# 张量
张量是一种特殊的数据结构，与数组和矩阵非常相似。在 PyTorch 中，我们使用张量对模型的输入和输出以及模型的参数进行编码。

张量类似于 NumPy 的 ndarrays，不同之处在于张量可以在 GPU 或其他硬件加速器上运行。事实上，张量和 NumPy 数组通常可以共享相同的底层内存，从而无需复制数据（请参阅 [Bridge with NumPy](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label)）。张量也针对自动微分进行了优化（我们将在稍后的 [Autograd](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) 部分中看到更多相关信息）。如果您熟悉 ndarrays，那么您将对 Tensor API 不陌生。如果不熟悉，那么就来学习吧！

In [1]:
import torch
import numpy as np

## 初始化一个张量
张量可以通过多种方式初始化。下面来看看几个例子：

**直接从原始数据初始化**
张量可以直接从数据中创建。数据类型是自动推断的。

In [2]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)

tensor([[1, 2],
        [3, 4]])


**从numpy数组初始化**
可以从 NumPy 数组创建张量（反之亦然 - 请参阅 [Bridge with NumPy](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label)）。

In [3]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)


**从其它张量初始化**
除非明确覆盖，否则新张量保留参数张量的属性（形状、数据类型）。

In [4]:
x_ones = torch.ones_like(x_data)
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.7045, 0.4677],
        [0.2551, 0.4118]]) 



**具有随机值或者常量值**
下面的shape是一个指定了张量的元组。在下面的函数中，它决定了输出张量的维度。

In [5]:
shape = (2, 3, )
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.1145, 0.6818, 0.1542],
        [0.0803, 0.7747, 0.9535]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


## 张量属性
张量属性描述了张量的维度、数据类型和存储设备

In [6]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## 张量的操作
[这里](https://pytorch.org/docs/stable/torch.html)全面描述了100多个张量操作，包括算术，线性代数，矩阵操作（转置，索引，切片），采样等。

每一个操作都可以在GPU上运行（通常比GPU更快）。如果您使用的是COLAB，请通过 运行时>更改运行时类型>GPU 操作链路分配GPU。

默认情况下都在CPU上创建张量。我们需要使用.to方法将张量显式移动到GPU（在检查GPU可用性之后）。请记住，在时间和内存方面，跨设备复制大量张量会非常损耗性能！

In [7]:
if torch.cuda.is_available():
    print("cuda available")
    tensor = tensor.to("cuda")

cuda available


从列表中尝试一些操作。如果您熟悉Numpy API，则会发现使用张量API轻而易举。

**标准的numpy型索引和切片**

In [8]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:, 1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
